In [ ]:
# Install necessary libraries
!pip install transformers unsloth axolotl torch datasets

# Import libraries
from transformers import QwenForCausalLM, QwenTokenizer
import torch
from datasets import load_dataset


In [ ]:
# Example: Use custom datasets or download a set of AI research papers
# Here we will assume that you have a dataset of papers

# For demonstration purposes, we can use HuggingFace's datasets like 'arxiv' or 'ai2_arc' (for research papers).
dataset = load_dataset('ai2_arc', 'long')

# Preprocess the dataset (tokenize and clean)
def preprocess_function(examples):
    return tokenizer(examples['question'], padding="max_length", truncation=True, max_length=512)

tokenizer = QwenTokenizer.from_pretrained("Qwen/Qwen2.5-3B")

train_dataset = dataset["train"].map(preprocess_function, batched=True)
val_dataset = dataset["validation"].map(preprocess_function, batched=True)
test_dataset = dataset["test"].map(preprocess_function, batched=True)

# Save processed datasets for future use
train_dataset.save_to_disk('train_dataset')
val_dataset.save_to_disk('val_dataset')
test_dataset.save_to_disk('test_dataset')


In [ ]:
from transformers import Trainer, TrainingArguments

# Load pre-trained model
model = QwenForCausalLM.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()


In [ ]:
from sklearn.metrics import accuracy_score

# Generate predictions on the test set
predictions = trainer.predict(test_dataset)

# Evaluate using accuracy
accuracy = accuracy_score(predictions.predictions.argmax(axis=-1), test_dataset["label"])
print(f"Test Accuracy: {accuracy}")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import pipeline

# Load quantized model (you would use a suitable quantization method)
model = QwenForCausalLM.from_pretrained("path_to_quantized_model")

# Save model as .gguf format
# The following is just an example, as actual .gguf quantization is specific to certain frameworks
model.save_pretrained("quantized_model.gguf")

# Inference pipeline
generator = pipeline('text-generation', model=model, tokenizer=model)

def generate_answer(question):
    return generator(question, max_length=100)[0]['generated_text']

# Test inference
question = "What is the impact of reinforcement learning in AI?"
answer = generate_answer(question)
print(answer)


In [ ]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Load a RAG model
rag_model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq")

# Define retriever
rag_tokenizer = RagTokenizer.from_pretrained("facebook/rag-tokenizer-nq")
rag_retriever = RagRetriever.from_pretrained("facebook/rag-tokenizer-nq")

# Inference with RAG
def rag_inference(question):
    inputs = rag_tokenizer(question, return_tensors="pt")
    outputs = rag_model.generate(input_ids=inputs["input_ids"])
    return rag_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test RAG
rag_answer = rag_inference("What is reinforcement learning?")
print(rag_answer)
